In [ ]:
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import io


store_columns = (
    "store_name",
    "branch",
    "area",
    "tel",
    "address",
    "latitude",
    "longitude",
    "category",
    "review_cnt",
    "big_cate",
    "mean_score",
    "store",
    "user",
    "score",
    "lifestyle",
    "ratio",
    "map_lifestyle",
    "detail_lifestyle"
)

data = []  # 음식점 테이블
with open('../data/lifestylegroup_userscore.csv') as f:
    for row in csv.DictReader(f):
        data.append(row)

stores = json.dumps(data)

stores = []  # 음식점 테이블

for d in data:

    # review_cnt = 
    stores.append(
        [
            d["store_name"],
            d["branch"],
            d["area"],
            d["tel"],
            d["address"],
            d["latitude"],
            d["longitude"],
            d["category"],
            d["review_cnt"],
            d["big_cate"],
            d["mean_score"],
            d["store"],
            d["lifestyle"],
            d["ratio"],
            d["map_lifestyle"],
            d["detail_lifestyle"]
        ]
    )

store_new = pd.DataFrame(data=stores, columns=store_columns)

print(store_new)

In [ ]:
# store_new.dtypes
store_new["store"].isnull()
# store_new["store"] = stores["store"].apply(pd.to_numeric)

In [ ]:
import json
import csv
import pandas as pd
import os
import shutil

DATA_DIR = "../data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")

store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도  # 위치기반 음식점 추천할 때 사용
    "longitude",  # 음식점 경도  # 위치기반 음식점 추천할 때 사용
    "category",  # 음식점 카테고리
    "review_cnt",  # 리뷰 갯수
)

menu_colums = (
    "store",  # 음식점 고유번호
    "menu",  # 메뉴 이름
    "price",  # 메뉴 가격
)

hour_colums = (
    "type",  # 영업시간 종류
    "week_type",  # 주단위 종류
    "store",  # 음식점 고유번호
    "mon",  # 월요일 포함유무
    "tue",  # 화요일 포함유무
    "wed",  # 수요일 포함유무
    "thu",  # 목요일 포함유무
    "fri",  # 금요일 포함유무
    "sat",  # 토요일 포함유무
    "sun",  # 일요일 포함유무
    "start_time",  # 시작시간
    "end_time",  # 종료시간
    "etc",  # 기타
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호
    "user",  # 유저 고유번호
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
)

user_colums = (
    "id",  # 유저 고유번호
    "gender",  # 성별
    "born_year"  # 태어난 해
)


def import_data(data_path=DATA_FILE):
    """
    Req. 1-1-1 음식점 데이터 파일을 읽어서 Pandas DataFrame 형태로 저장합니다
    """
    

    try:
        with open(data_path, encoding="utf-8") as f:
            data = json.loads(f.read())
    except FileNotFoundError:
        print(f"`{data_path}` 가 존재하지 않습니다.")
        exit(1)

    stores = []  # 음식점 테이블
    menus = []  # 메뉴 테이블
    hours = []  # 음식점 시간 테이블
    reviews = []  # 리뷰 테이블
    users = []  # 유저 테이블

    for d in data:

        categories = [c["category"] for c in d["category_list"]]
        # review_cnt = 
        stores.append(
            [
                d["id"],
                d["name"],
                d["branch"],
                d["area"],
                d["tel"],
                d["address"],
                d["latitude"],
                d["longitude"],
                "|".join(categories),
                d["review_cnt"]
            ]
        )
  
        for menu in d["menu_list"]:
            menus.append(
                [d["id"], menu["menu"], menu["price"]]
            )
        
        for hour in d["bhour_list"]:
            hours.append(
                [
                    hour["type"], hour["week_type"], d["id"],
                    hour["mon"], hour["tue"], hour["wed"], hour["thu"],
                    hour["fri"], hour["sat"], hour["sun"],
                    hour["start_time"], hour["end_time"], hour["etc"]
                ]
            )
  
        for review in d["review_list"]:
            r = review["review_info"]
            u = review["writer_info"]

            reviews.append(
                [r["id"], d["id"], u["id"], r["score"], r["content"], r["reg_time"]]
            )

            users.append(
                [u["id"], u["gender"], u["born_year"]]
            )

    store_frame = pd.DataFrame(data=stores, columns=store_columns)
    menu_frame = pd.DataFrame(data=menus, columns=menu_colums)
    hour_frame = pd.DataFrame(data=hours, columns=hour_colums)
    review_frame = pd.DataFrame(data=reviews, columns=review_columns)
    user_frame = pd.DataFrame(data=users, columns=user_colums)
    user_frame = user_frame.drop_duplicates()

    return {"stores": store_frame, "menus": menu_frame, "hours": hour_frame,
            "reviews": review_frame, "users": user_frame}


def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)


def load_dataframes():
    return pd.read_pickle(DUMP_FILE)


print("[*] Parsing data...")
data = import_data()
print("[+] Done")

print("[*] Dumping data...")
dump_dataframes(data)
print("[+] Done\n")

data = load_dataframes()

term_w = shutil.get_terminal_size()[0] - 1
separater = "-" * term_w

# print("[메뉴]")
# print(f"{separater}\n")
# print(data["menus"].head())
# print(f"\n{separater}\n\n")

# print("[영업시간]")
# print(f"{separater}\n")
# print(data["hours"].head())
# print(f"\n{separater}\n\n")

# print("[리뷰]")
# print(f"{separater}\n")
# print(data["reviews"].head())
# print(f"\n{separater}\n\n")

# print("[유저]")
# print(f"{separater}\n")
# print(data["users"].head())
# print(f"\n{separater}\n\n")

print(data["menus"])

In [ ]:
import pickle

f = open("../data/lifestylegroupuser.pickle", "rb")
tokendata = pickle.load(f)
f.close()

data.to_dict(f)

In [ ]:
# 가격가지고올때, 메뉴가 하나의 가게에 여러개 나오니까 그거로 평균내서 store랑 합치기
menu = data["menus"]
menus = menu.groupby("store").mean()
# stores.to_excel("../data/store_big_cate.xlsx")
menus.to_excel("../data/menu_check.xlsx")
# menus = menu.groupby("store").mean()
print(menus.head())
# print(menus)
# store_new["store"] = stores["store"].apply(pd.to_numeric)
# store_menu = pd.merge(store_new, menus, left_on="store", right_on="store")
# print(store_menu.head())

In [ ]:
def get_similar_movies(movie_title, n_ratings_filter=100, n_recommendations=5):
    similar = matrix.corrwith(matrix[movie_title])
    corr_similar = pd.DataFrame(similar, columns=['correlation'])
    corr_similar.dropna(inplace=True)
    
    orig = data.copy()
    
    corr_with_movie = pd.merge(
        left=corr_similar, 
        right=orig, 
        on='title')[['title', 'correlation', 'numRatings']].drop_duplicates().reset_index(drop=True)
    
    result = corr_with_movie[corr_with_movie['numRatings'] > n_ratings_filter].sort_values(by='correlation', ascending=False)
    
    return result.head(n_recommendations)